In [28]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
import torch
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cpu"
model_name_or_path = "bigscience/mt0-small"
tokenizer_name_or_path = "bigscience/mt0-small"

checkpoint_name = "financial_sentiment_analysis_lora_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-3
num_epochs = 3
batch_size = 8

In [29]:
# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, trust_remote_code=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   1 # creating model                                                                             │
│   2 peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lor     │
│   3                                                                                              │
│ ❱ 4 model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True)     │
│   5 model = get_peft_model(model, peft_config)                                                   │
│   6 model.print_trainable_parameters()                                                           │
│   7 model                                                                                        │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\transformers\models\auto\auto_factory.py:466 in     │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   463 │   │   │   │   pretrained_model_name_or_path, module_file + ".py", class_name, **hub_kw   │
│   464 │   │   │   )                                                                              │
│   465 │   │   │   model_class.register_for_auto_class(cls.__name__)                              │
│ ❱ 466 │   │   │   return model_class.from_pretrained(                                            │
│   467 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   468 │   │   │   )                                                                              │
│   469 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\transformers\modeling_utils.py:2541 in              │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   2538 │   │   # We'll need to download and cache each checkpoint shard if the checkpoint is sh  │
│   2539 │   │   if is_sharded:                                                                    │
│   2540 │   │   │   # rsolved_archive_file becomes a list of files that point to the different c  │
│ ❱ 2541 │   │   │   resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(         │
│   2542 │   │   │   │   pretrained_model_name_or_path,                                            │
│   2543 │   │   │   │   resolved_archive_file,                                                    │
│   2544 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\transformers\utils\hub.py:925 in                    │
│ get_checkpoint_shard_files                                                                       │
│                                                                                                  │
│    922 │   for shard_filename in tqdm(shard_filenames, desc="Downloading shards", disable=not s  │
│    923 │   │   try:                                                                              │
│    924 │   │   │   # Load from URL                                                               │
│ ❱  925 │   │   │   cached_filename = cached_file(          

In [ ]:
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Found cached dataset financial_phrasebank (C:/Users/MSI/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': 'Revenue grew 12 percent to (  x20ac ) 3.6 billion ( US$ 4.5 billion ) .',
 'label': 2,
 'text_label': 'positive'}

In [ ]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Using pad_token, but it is not set yet.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   13 │   return model_inputs                                                                     │
│   14                                                                                             │
│   15                                                                                             │
│ ❱ 16 processed_datasets = dataset.map(                                                           │
│   17 │   preprocess_function,                                                                    │
│   18 │   batched=True,                                                                           │
│   19 │   num_proc=1,                                                                             │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\datasets\dataset_dict.py:851 in map                 │
│                                                                                                  │
│    848 │   │   if cache_file_names is None:                                                      │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│ ❱  851 │   │   │   {                                                                             │
│    852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\datasets\dataset_dict.py:852 in <dictcomp>          │
│                                                                                                  │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│    851 │   │   │   {                                                                             │
│ ❱  852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│    855 │   │   │   │   │   with_rank=with_rank,                                                  │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\datasets\arrow_dataset.py:578 in wrapper            │
│                                                                                                  │
│    575 │   │   else:                                                                             │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    577 │   │   # apply actual function                                                           │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    580 │   │   for dataset in datasets:                                                          │
│    581 │   │   │   # Remove task templates if a column mapp

In [ ]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   1 # optimizer and lr scheduler                                                                 │
│ ❱ 2 optimizer = torch.optim.AdamW(model.parameters(), lr=lr)                                     │
│   3 lr_scheduler = get_linear_schedule_with_warmup(                                              │
│   4 │   optimizer=optimizer,                                                                     │
│   5 │   num_warmup_steps=0,                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│    1 # training and evaluation                                                                   │
│ ❱  2 model = model.to(device)                                                                    │
│    3                                                                                             │
│    4 for epoch in range(num_epochs):                                                             │
│    5 │   model.train()                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│    1 # print accuracy                                                                            │
│    2 correct = 0                                                                                 │
│    3 total = 0                                                                                   │
│ ❱  4 for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):                     │
│    5 │   if pred.strip() == true.strip():                                                        │
│    6 │   │   correct += 1                                                                        │
│    7 │   total += 1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'eval_preds' is not defined

In [ ]:
!echo $accuracy

$accuracy


In [ ]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   1 # saving model                                                                               │
│   2 peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"      │
│ ❱ 3 model.save_pretrained(peft_model_id)                                                         │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
ckpt = f"{peft_model_id}/adapter_model.bin"
!du -h $ckpt

'du' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\MSI\Downloads\env\lib\site-packages\huggingface_hub\utils\_errors.py:259 in             │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\requests\models.py:1021 in raise_for_status         │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: 
https://huggingface.co/tiiuae/falcon-7b_LORA_SEQ_2_SEQ_LM/resolve/main/adapter_config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\MSI\Downloads\env\lib\site-packages\peft\utils\config.py:106 in from_pretrained         │
│                                                                                                  │
│   103 │   │   │   config_file = os.path.join(path, CONFIG_NAME)                                  │
│   104 │   │   else:                                                                              │
│   105 │   │   │   try:                                                                           │
│ ❱ 106 │   │   │   │   config_file = hf_hub_download(pretrained_model_name_or_path, CONFIG_NAME   │
│   107 │   │   │   except Exception:                                                              │
│   108 │   │   │   │   raise ValueError(f"Can't find '{CONFIG_NAME}' at '{pretrained_model_name   │
│   109                                                                                            │
│                                                                                                  │
│ c:\Users\MSI\Downloads\env\lib\site-packages\huggingface_hub\utils\_validators.py:120 in         │
│ _inner_fn                                                                                        │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_

In [ ]:
model.eval()
inputs = tokenizer("""Insurance motto: "Who is trying nothing, has nothing!"So they prefer to be punished by the courts and pay with compensation, rather than doing what they are commissioned for: means and results on a lasting recovery of disorders! It is actually unhappy, but from the principle that we know their operation….We act accordingly for our customers 💪bravo to you ... See more	""", return_tensors="pt")
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 model.eval()                                                                                 │
│   2 inputs = tokenizer("""Insurance motto: "Who is trying nothing, has nothing!"So they pref     │
│   3 print(inputs)                                                                                │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
def classify(x):
    inputs = tokenizer(x, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0].title()